In [1]:
import numpy as np
from scipy import ndimage
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import gaussian_process
from sklearn.cross_validation import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from itertools import izip
import csv

In [2]:
n_dependencies = 2
num_images = 319
images = np.empty([1,20000])

In [3]:
for i in range(1,num_images+1):
    filename = "gray_images_all/image" + str(i) + ".jpg"
    img = ndimage.imread(filename)
    if (i == 1):
        images = img.flatten()
    else:
        images = np.vstack((images, img.flatten()))
        
images = images.astype(float)
print images.shape
print images.min()
print images.max()

(319L, 20000L)
0.0
255.0


In [4]:
scaler = preprocessing.StandardScaler().fit(images)
images_scaled = scaler.transform(images)
print images_scaled.shape
print images_scaled.min()
print images_scaled.max()

(319L, 20000L)
-17.8325545001
17.8325545001


In [5]:
thresh_components = 10
n_components = thresh_components
while(True):
    pca0 = decomposition.PCA(n_components=n_components)
    pca0.fit(images_scaled)
    if pca0.explained_variance_ratio_.sum() > 0.9:
        break
    n_components += thresh_components

In [6]:
pca = decomposition.PCA(n_components=n_components)
pca.fit(images_scaled)
data = pca.transform(images_scaled)
print data.shape
print data.min()
print data.max()

(319L, 100L)
-102.788361097
202.391869066


In [7]:
print n_components
print pca.explained_variance_ratio_.sum()

100
0.902236779604


In [8]:
for i in range(len(data)-n_dependencies):
    newrow = []
    for j in range(n_dependencies):
        newrow = np.hstack((newrow, data[i+j]))
    if (i == 0):
        X = newrow
    else:
        X = np.vstack((X, newrow))

In [9]:
print 'X :',X.shape
ylist = []
for i in range(n_dependencies,len(data)):
    ylist.append(data[i])
y = np.asarray(ylist)
print 'y :', y.shape

X : (317L, 200L)
y : (317L, 100L)


In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=0)
test_size = 0.4
if X.shape[0] == y.shape[0]:
    split = int(round((1-test_size)*X.shape[0]))
    X_train, X_test = X[:split], X[split:X.shape[0]]
    y_train, y_test = y[:split], y[split:y.shape[0]]
    print X_train.shape
    print y_train.shape
    print X_test.shape
    print y_test.shape
else:
    print 'X, y shapes do not match'

(190L, 200L)
(190L, 100L)
(127L, 200L)
(127L, 100L)


In [11]:
gp = gaussian_process.GaussianProcess(regr='constant',
                                      corr='squared_exponential',
                                      theta0=2e-2,
                                      thetaL=1e-4,
                                      thetaU=1e-1)
gp.fit(X_train, y_train)

GaussianProcess(beta0=None,
        corr=<function squared_exponential at 0x0000000015DAEEB8>,
        normalize=True, nugget=array(2.220446049250313e-15),
        optimizer='fmin_cobyla', random_start=1,
        random_state=<mtrand.RandomState object at 0x0000000003AF3B70>,
        regr=<function constant at 0x0000000015DAEAC8>,
        storage_mode='full', theta0=array([[ 0.02]]),
        thetaL=array([[ 0.0001]]), thetaU=array([[ 0.1]]), verbose=False)

In [12]:
print X_test[0].shape
print X_test[0][:pca.n_components_].shape

(200L,)
(100L,)


In [13]:
y_pred_list=[]
sigma = []

for i in range(len(X_test)-n_dependencies):
    if(i == 0):
        X_test_sample = X_test[0]
    else:
        end = (n_dependencies-1)*pca.n_components_
        X_test_sample = np.hstack((X_test[i][:end], y_test_pred[0]))
    y_test_pred, mse_pred = gp.predict(X_test_sample, eval_MSE=True)
    y_pred_list.append(y_test_pred[0])
    sigma.append(np.sqrt(mse_pred))

y_pred = np.asarray(y_pred_list)
y_pred_size = y_pred.shape[0]
print y_pred.shape
print y_test.shape

(125L, 100L)
(127L, 100L)


In [14]:
inv_y_pred = pca.inverse_transform(y_pred)
print inv_y_pred.shape
inv_y_test = pca.inverse_transform(y_test)
print inv_y_test.shape

(125L, 20000L)
(127L, 20000L)


In [15]:
transformed_y_pred = scaler.inverse_transform(inv_y_pred)
original_y_test = scaler.inverse_transform(inv_y_test[n_dependencies:])
print transformed_y_pred.shape
print original_y_test.shape

(125L, 20000L)
(125L, 20000L)


In [16]:
print 'inverted :', inv_y_pred[0].min(), inv_y_pred[0].max()
print 'transformed :', transformed_y_pred[0].min(), transformed_y_pred[0].max()

inverted : -2.24682706503 4.45862238429
transformed : -3.79035777517 256.919916037


In [17]:
pred_images = transformed_y_pred.reshape((y_pred_size, 100, 200))
#original_image = original_y_test.reshape((y_pred_size, 100, 200))
original_images = images[-y_pred_size:].reshape((y_pred_size, 100, 200))

In [18]:
print pred_images[0].shape
print original_images[0].shape

(100L, 200L)
(100L, 200L)


In [19]:
rmse_lds = np.asarray(sigma)
print rmse_lds.shape
print 'dependecies :', n_dependencies
print 'min rmse_lds:', rmse_lds.min()
print 'max rmse_lds:', rmse_lds.max()
print 'mean rmse_lds:', rmse_lds.mean()

(125L, 1L)
dependecies : 2
min rmse_lds: 2.5878767321
max rmse_lds: 12.8153381633
mean rmse_lds: 9.75738999601


In [25]:
mmscalar = preprocessing.MinMaxScaler((0,255))
images = []
for pred_img in pred_images:
    images.append(mmscalar.fit_transform(pred_img))
output_images = np.asarray(images).reshape((y_pred_size, 100, 200))
print 'output images :', output_images.min(), output_images.max()
print 'original images :', original_images.min(), original_images.max()

output images : 0.0 255.0
original images : 0.0 255.0


In [26]:
rmse_hds_list = []
for pred_img, orig_img in izip(output_images, original_images):
    rmse_hds_list.append(mean_squared_error(orig_img, pred_img)**0.5)
rmse_hds = np.asarray(rmse_hds_list)
print rmse_hds.shape
print 'dependecies :', n_dependencies
print 'min rmse_hds:', rmse_hds.min()
print 'max rmse_hds:', rmse_hds.max()
print 'mean rmse_hds:', rmse_hds.mean()

(125L,)
dependecies : 2
min rmse_hds: 47.5880840043
max rmse_hds: 82.7111174207
mean rmse_hds: 71.2188655256


In [ ]:
info = [n_dependencies, rmse.min(), rmse.max(), rmse.mean()]
with open('rmse_info.csv','a') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(info)